# Find the Repos Available in your Database, and What Repository Groups They Are In

## Connect to your database

In [1]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
import json
warnings.filterwarnings('ignore')

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

### Retrieve Available Respositories

In [2]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SELECT a.rg_name,
                a.repo_group_id,
                b.repo_name,
                b.repo_id,
                b.forked_from,
                b.repo_archived 
            FROM
                repo_groups a,
                repo b 
            WHERE
                a.repo_group_id = b.repo_group_id 
            ORDER BY
                rg_name,
                repo_name;   

    """)

repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes

,rg_name,repo_group_id,repo_name,repo_id,forked_from,repo_archived
0,chaoss,25155,augur,25440,Parent not available,0
1,chaoss,25155,augur-license,25460,Parent not available,0
2,chaoss,25155,community-reports,25558,Parent not available,0
3,chaoss,25155,governance,25449,Parent not available,0
4,chaoss,25155,grimoirelab,25448,Parent not available,0
...,...,...,...,...,...,...
75,SaltStack,60017,salt-winrepo-ng,30356,Parent not available,0
76,SaltStack,60017,salty-vagrant,30362,Parent not available,0
77,SaltStack,60017,sublime-text,30347,Parent not available,0
78,SaltStack,60017,takara,30380,Parent not available,1


rg_name          object
repo_group_id     int64
repo_name        object
repo_id           int64
forked_from      object
repo_archived     int64
dtype: object

### Create a Simpler List for quickly Identifying repo_group_id's and repo_id's for other queries

In [3]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SELECT b.repo_id,
                a.repo_group_id,
                b.repo_name,
                a.rg_name
            FROM
                repo_groups a,
                repo b 
            WHERE
                a.repo_group_id = b.repo_group_id 
            ORDER BY
                rg_name,
                repo_name;   

    """)

repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes

,repo_id,repo_group_id,repo_name,rg_name
0,25440,25155,augur,chaoss
1,25460,25155,augur-license,chaoss
2,25558,25155,community-reports,chaoss
3,25449,25155,governance,chaoss
4,25448,25155,grimoirelab,chaoss
...,...,...,...,...
75,30356,60017,salt-winrepo-ng,SaltStack
76,30362,60017,salty-vagrant,SaltStack
77,30347,60017,sublime-text,SaltStack
78,30380,60017,takara,SaltStack


repo_id           int64
repo_group_id     int64
repo_name        object
rg_name          object
dtype: object

In [4]:
laborlist = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SELECT DATE,
                    repo_name,
                    rg_name,
                    round( SUM ( estimated_labor_hours ), 2 ) AS estimated_hours 
                FROM
                    (
                    SELECT
                        date_trunc('month', max(repo_labor.rl_analysis_date)::TIMESTAMP::DATE) as date, 
                        repo.repo_id,
                        repo_name,
                        rg_name,
                        programming_language,
                        MAX ( repo_labor.rl_analysis_date) as analysis_date,
                        SUM ( total_lines ) AS repo_total_lines,
                        SUM ( code_lines ) AS repo_code_lines,
                        SUM ( comment_lines ) AS repo_comment_lines,
                        SUM ( blank_lines ) AS repo_blank_lines,
                        AVG ( code_complexity ) AS repo_lang_avg_code_complexity,
                        AVG ( code_complexity ) * SUM ( code_lines ) + 20 AS estimated_labor_hours 
                    FROM
                        repo_labor,
                        repo,
                        repo_groups 
                    WHERE
                        repo.repo_id = 22095
                        AND repo.repo_group_id = repo_groups.repo_group_id
                    GROUP BY
                        repo.repo_id,
                        rg_name,
                        programming_language,
                        repo_name 
                    ORDER BY
                        repo_name,
                        repo.repo_id,
                        programming_language 
                    ) C 
                GROUP BY
                    DATE,
                    repo_id,
                    repo_name, 
                    rg_name 
                ORDER BY
                    repo_name

    """)

laborlist = pd.read_sql(repo_query, con=engine)

display(laborlist)

laborlist.dtypes

,date,repo_name,rg_name,estimated_hours


date               object
repo_name          object
rg_name            object
estimated_hours    object
dtype: object